In [72]:
import networkx as nx
import numpy as np
import string
import matplotlib.pyplot as plt
import osmnx as ox
import pandas as pd
import pickle

In [73]:
city = 'Utrecht'
city_str = 'Utrecht, Netherlands'
dist = 6500

# city = 'Winterswijk'
# city_str = 'Winterswijk, Netherlands'
# dist = 10000

# city = 'Manhattan'
# city_str = 'Union Square, New York, United States'
# dist = 5000

In [74]:
filepath=f"../graphs/{city}.graph.graphml"
# ox.save_graph_geopackage(G, filepath=filepath)
G = ox.load_graphml(filepath=filepath)

with open(f'../data/escape_nodes_{city}.pkl', 'rb') as f:
    escape_nodes = pickle.load(f)
    
with open(f'../data/fugitive_start_{city}.pkl', 'rb') as f:
    fugitive_start = pickle.load(f)

if city != 'Winterswijk':    
    with open(f'../data/cameras_{city}.pkl', 'rb') as f:
        cameras = pickle.load(f)
elif city == 'Winterswijk':
    cameras = []

In [75]:
len(escape_nodes)

148

In [76]:
escape_nodes_filtered = []
for escape_node in escape_nodes:
    if G.has_node(escape_node):
        escape_nodes_filtered.append(escape_node)

with open('../data/escape_nodes_Utrecht_filtered.pkl', 'wb') as f: 
    pickle.dump(escape_nodes_filtered, f)

In [77]:
len(escape_nodes_filtered)

125

In [78]:
escape_nodes = escape_nodes_filtered.copy()

In [79]:
def draw_edges(graph):
    edges_fugitive = []

    # for i_r, route_time in enumerate(fugitive_routes):
    #     route = list(route_time.values())
    #     for i, node in enumerate(route):
    #         if i ==0:
    #             continue
    #         else:
    #             edges_fugitive1 = [(route[i], route[i-1])]
    #             edges_fugitive2 = [(route[i-1], route[i])]
    #             edges_fugitive.extend(tuple(edges_fugitive1))
    #             edges_fugitive.extend(tuple(edges_fugitive2))

    edge_colormap = ['lightgray'] * len(graph.edges())
    edge_weightmap = [1] * len(graph.edges())
    for index, edge in enumerate(graph.edges()):
        if edge in edges_fugitive:
            edge_colormap[index] = 'tab:orange'
            edge_weightmap[index] = 2

    return edge_colormap, edge_weightmap

In [80]:
def draw_nodes(G, fugitive_start, cameras, trafficlights, roundabouts, tunnels, bridges, escape_nodes):
    node_size = []
    node_color = []
    for node in G.nodes:
        if node == fugitive_start:
            node_size.append(40)
            node_color.append('tab:orange')
        elif node in escape_nodes:
            node_size.append(40)
            node_color.append('tab:red')
        elif node in cameras:
            node_size.append(10)
            node_color.append('tab:green')      
        elif node in trafficlights:
            node_size.append(10)
            node_color.append('purple')    
        elif node in roundabouts:
            node_size.append(10)
            node_color.append('gold')    
        elif node in tunnels:
            node_size.append(10)
            node_color.append('aqua')   
        elif node in bridges:
            node_size.append(10)
            node_color.append('royalblue')   
        else:
            node_size.append(0)
            node_color.append('lightgray')
    return node_size, node_color

In [81]:
from matplotlib.lines import Line2D

def show_graph(G, escape_nodes, fugitive_start, cameras, trafficlights, roundabouts, bridges, tunnels, save=False):
    # filepath=f"graphs/FLEE/Graph_FLEE.graph.graphml"
    # G = ox.load_graphml(filepath=filepath)

    edge_colormap, edge_weightmap = draw_edges(G)
    node_size, node_color = draw_nodes(G, fugitive_start, cameras, trafficlights, roundabouts, tunnels, bridges, escape_nodes)

    fig, ax = ox.plot_graph(
                G, bgcolor="white", node_color=node_color, node_size=node_size, edge_linewidth=edge_weightmap, edge_color=edge_colormap,
                )
    if save:

        custom_lines = [Line2D([0], [0], color='tab:orange', marker='o', markersize=10, linestyle=''),
                        Line2D([0], [0], color='tab:red', marker='o', markersize=10, linestyle=''),
                        Line2D([0], [0], color='tab:green', marker='o', markersize=10, linestyle=''),
                        Line2D([0], [0], color='purple', marker='o', markersize=10, linestyle=''),
                        Line2D([0], [0], color='gold', marker='o', markersize=10, linestyle=''),
                        Line2D([0], [0], color='aqua', marker='o', markersize=10, linestyle=''),
                        Line2D([0], [0], color='royalblue', marker='o', markersize=10, linestyle=''),
                        ]
        labels = ['start fugitive', 'escape node', 'camera', 'traffic light', 'roundabout', 'tunnel', 'bridge']
        ax.legend(custom_lines, labels, bbox_to_anchor=(1.1, 1.05))
        fig.savefig(f'graphs/{city}.png', bbox_inches='tight', dpi=300)

In [82]:
# show_graph(G, escape_nodes, fugitive_start, cameras, [], [], [], [], save=False)

In [83]:
roundabouts = []
for u,v, data in G.edges(data=True): 
    if 'junction' in data.keys():
        if data['junction'] == 'roundabout':
            roundabouts.append(u)
            roundabouts.append(v)
            
roundabouts = np.unique(roundabouts)

In [84]:
#for traffic lights: 
# 1) import non-simplified graph
# 2) find all traffic lights and save lat/lon
# 3) find nearest node for each
# 4) add 'traffic_signals' tag

In [85]:
import networkx as nx
import osmnx as ox

useful_tags_path = ['bridge', 'tunnel', 'oneway', 'lanes', 'ref', 'name', 'length',
                    'highway', 'maxspeed', 'service', 'access', 'area', 'name'
                    'landuse', 'width', 'est_width', 'junction', 'surface', 
                    'traffic_signals', 'crossing']

ox.settings.useful_tags_way=useful_tags_path

def generate_streetnetwork(city_str, dist):
    """
    Load a network from OSMNX which will be used for the simulation runs. The generated network is from Rotterdam.
    Returns
    -------
    G: network MultiGraph
        Network Graph with different attributes:
            Nodes and Edges with LineString with coordinates,
            Edges have properties Speed, Travel time, Length.

    """
    lonlat=     'Netherlands'                                 #coordinates of the city centre of Rotterdam
    roadtype=   "drive"                                     #select only roads which are driveable by the car.

    cf = '["highway"~"motorway|motorway_link|trunk|trunk_link|primary|secondary|tertiary|residential"]'

    ox.settings.log_console=True
    ox.settings.use_cache=True

    # download street network data from OSM and construct a MultiDiGraph model
    G = ox.graph_from_address(city_str, dist=dist, network_type='drive', simplify=False, custom_filter=cf)

    return G

G_notsimplified=generate_streetnetwork(city_str, dist)

In [86]:
if city == 'Manhattan':
    G_notsimplified = G.copy()
    
trafficlight_locs = []
for u, data in G_notsimplified.nodes(data=True): 
    if 'traffic_signals' in data.keys():
        trafficlight_locs.append(data)
    elif 'highway' in data.keys():
        if data['highway'] == 'traffic_signals':
            trafficlight_locs.append(data)
    elif 'crossing' in data.keys():
        if data['crossing'] == 'traffic_signals':
            trafficlight_locs.append(data)

In [87]:
trafficlight_locs

[{'y': 52.0348192,
  'x': 5.0770202,
  'highway': 'traffic_signals',
  'street_count': 3},
 {'y': 52.0380178,
  'x': 5.1508766,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.0381438,
  'x': 5.1513037,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.0408769,
  'x': 5.1678242,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.046292,
  'x': 5.1178136,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.0562605,
  'x': 5.1201704,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.056391,
  'x': 5.1205257,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.05675,
  'x': 5.1243359,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.0575811,
  'x': 5.1224603,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.0577659,
  'x': 5.1222054,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'y': 52.0591971,
  'x': 5.0964093,
  'highway': 'traffic_signals',
  'street_count': 2},
 {'

In [88]:
trafficlights = []
for trafficlight in trafficlight_locs:
    closest_node = ox.nearest_nodes(G, trafficlight['x'], trafficlight['y'])
    trafficlights.append(closest_node)
    
trafficlights = np.unique(trafficlights)

In [89]:
len(trafficlights)

421

In [90]:
tunnels = []
for u, v, data in G.edges(data=True): 
    if 'tunnel' in data.keys():
        tunnels.append(u)
        tunnels.append(v)
            
tunnels = np.unique(tunnels)

In [91]:
bridges = []
for u, v, data in G.edges(data=True): 
    if 'bridge' in data.keys():
        if data['bridge'] == 'movable':
            bridges.append(u)
            bridges.append(v)
            
bridges = np.unique(bridges)
bridges

array([  45099870,   45100303,   45100812,   45101704,   45140936,
         45143415,   45145918,   45147197,   45154789,   45209943,
         45210358,   45214871,   45228886,   45235580,   45236404,
         45244963,   45269577,   45270855,   45327028,   45364855,
       2275251614, 3163236364, 3669833141, 4250084614, 6283257130,
       7644557070, 7644587794, 8770170705], dtype=int64)

In [92]:
# show_graph(G, escape_nodes, fugitive_start, cameras, trafficlights, roundabouts, bridges, tunnels, save=False)

Loop through all edges in graph. If u/v has camera/traffic light/roundabout/etc, add time to travel_time of edge. e.g. 30 sec for camera, 10 sec for traffic light, 5 for roundabout (bc multiple nodes)?

In [93]:
def set_travel_time_adj(G, 
                        cameras, camera_extra_time, 
                        trafficlights, trafficlight_extra_time,
                        roundabouts, roundabout_extra_time,
                        tunnels, tunnel_extra_time,
                        bridges, bridge_extra_time
                        ):
    travel_time_dict = {}
    for u, v, data in G.edges(data=True): 
        travel_time_adj = data['travel_time']
        
        if data['highway'] in ['motorway', 'trunk', 'primary']: 
            travel_time_adj += 40
        
        if u in cameras:
            travel_time_adj += camera_extra_time
        if u in trafficlights:
            travel_time_adj += trafficlight_extra_time
        if u in roundabouts:
            travel_time_adj += roundabout_extra_time
        if u in tunnels:
            travel_time_adj += tunnel_extra_time
        if u in bridges:
            travel_time_adj += bridge_extra_time
        
        if v in cameras:
            travel_time_adj += camera_extra_time
        if v in trafficlights:
            travel_time_adj += trafficlight_extra_time
        if v in roundabouts:
            travel_time_adj += roundabout_extra_time
        if v in tunnels:
            travel_time_adj += tunnel_extra_time
        if v in bridges:
            travel_time_adj += bridge_extra_time
        
        travel_time_dict[(u, v, 0)] = float(travel_time_adj)
    
    nx.set_edge_attributes(G, travel_time_dict, "travel_time_adj")

In [94]:
G_saved = G.copy()

# Cool

In [95]:
camera_extra_time = 30
roundabout_extra_time = 5
trafficlight_extra_time = 10
tunnel_extra_time = 5
bridge_extra_time = 0

G = G_saved.copy()
set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )

filepath=f"../graphs/sensitivity_analysis/{city}_cool_default.graph.graphml"
# ox.save_graph_geopackage(G, filepath=filepath)
ox.save_graphml(G,filepath)

route_fugitive = []
while len(route_fugitive) < 1000:
    for escape_node in escape_nodes_filtered:
        try:
            path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
            # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
            route_fugitive.append(path)
        except:
            continue

with open(f'../data/sensitivity_analysis/escape_routes_{city}_cool_default.pkl', 'wb') as f: 
    pickle.dump(route_fugitive, f)

In [96]:
camera_extra_time = 30
roundabout_extra_time = 5
trafficlight_extra_time = 10
tunnel_extra_time = 5
bridge_extra_time = 0

for camera_extra_time in [20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_cool_camera_{camera_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_cool_camera_{camera_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [97]:
camera_extra_time = 30
roundabout_extra_time = 5
trafficlight_extra_time = 10
tunnel_extra_time = 5
bridge_extra_time = 0

for roundabout_extra_time in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_cool_roundabout_{roundabout_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_cool_roundabout_{roundabout_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [98]:
camera_extra_time = 30
roundabout_extra_time = 5
trafficlight_extra_time = 10
tunnel_extra_time = 5
bridge_extra_time = 0

for trafficlight_extra_time in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_cool_trafficlight_{trafficlight_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_cool_trafficlight_{trafficlight_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [99]:
camera_extra_time = 30
roundabout_extra_time = 5
trafficlight_extra_time = 10
tunnel_extra_time = 5
bridge_extra_time = 0

for tunnel_extra_time in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_cool_tunnel_{tunnel_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_cool_tunnel_{tunnel_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [100]:
camera_extra_time = 30
roundabout_extra_time = 5
trafficlight_extra_time = 10
tunnel_extra_time = 5
bridge_extra_time = 0

for bridge_extra_time in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_cool_bridge_{bridge_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_cool_bridge_{bridge_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


# Hot

In [101]:
camera_extra_time = 0
roundabout_extra_time = 5
trafficlight_extra_time = 20
tunnel_extra_time = 10
bridge_extra_time = 0

G = G_saved.copy()
set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )

filepath=f"../graphs/sensitivity_analysis/{city}_hot_default.graph.graphml"
# ox.save_graph_geopackage(G, filepath=filepath)
ox.save_graphml(G,filepath)

route_fugitive = []
while len(route_fugitive) < 1000:
    for escape_node in escape_nodes_filtered:
        try:
            path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
            # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
            route_fugitive.append(path)
        except:
            continue

with open(f'../data/sensitivity_analysis/escape_routes_{city}_hot_default.pkl', 'wb') as f: 
    pickle.dump(route_fugitive, f)

In [102]:
camera_extra_time = 0
roundabout_extra_time = 5
trafficlight_extra_time = 20
tunnel_extra_time = 10
bridge_extra_time = 0

for roundabout_extra_time in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_hot_roundabout_{roundabout_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_hot_roundabout_{roundabout_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [103]:
camera_extra_time = 0
roundabout_extra_time = 5
trafficlight_extra_time = 20
tunnel_extra_time = 10
bridge_extra_time = 0

for trafficlight_extra_time in [10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_hot_trafficlight_{trafficlight_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_hot_trafficlight_{trafficlight_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [104]:
camera_extra_time = 0
roundabout_extra_time = 5
trafficlight_extra_time = 20
tunnel_extra_time = 10
bridge_extra_time = 0

for tunnel_extra_time in [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_hot_tunnel_{tunnel_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_hot_tunnel_{tunnel_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [105]:
camera_extra_time = 0
roundabout_extra_time = 5
trafficlight_extra_time = 20
tunnel_extra_time = 10
bridge_extra_time = 0

for bridge_extra_time in [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_hot_bridge_{bridge_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_hot_bridge_{bridge_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020


In [106]:
camera_extra_time = 0
roundabout_extra_time = 5
trafficlight_extra_time = 20
tunnel_extra_time = 10
bridge_extra_time = 0

for camera_extra_time in [0]:
    G = G_saved.copy()
    set_travel_time_adj(G, 
                    cameras, camera_extra_time, 
                    trafficlights, trafficlight_extra_time,
                    roundabouts, roundabout_extra_time,
                    tunnels, tunnel_extra_time,
                    bridges, bridge_extra_time
                    )
    
    filepath=f"../graphs/sensitivity_analysis/{city}_hot_camera_{camera_extra_time}.graph.graphml"
    # ox.save_graph_geopackage(G, filepath=filepath)
    ox.save_graphml(G,filepath)
    
    route_fugitive = []
    while len(route_fugitive) < 1000:
        for escape_node in escape_nodes_filtered:
            try:
                path = nx.shortest_path(G, fugitive_start, escape_node, 'travel_time_adj')
                # [escape_routes.append(path) for path in nx.all_simple_paths(G, fugitive_start, escape_node)]
                route_fugitive.append(path)
            except:
                continue
    
    with open(f'../data/sensitivity_analysis/escape_routes_{city}_hot_camera_{camera_extra_time}.pkl', 'wb') as f: 
        pickle.dump(route_fugitive, f)
    
    print(len(route_fugitive))

1020
